In [13]:
import itertools
import random

In [14]:
lens = [25, 30]
methods = ["norsnet", "bval", "ucon"]

# lens = [25]
# methods = ["norsnet"]

In [15]:
def write_files(out, files, files2=None):
    all_files = zip(itertools.repeat(1), files)
    if files2 is not None:
        all_files = itertools.chain(all_files, zip(itertools.repeat(2), files2))
    ids = []
    with open(out + ".idfile", 'w') as idfile:
        with open(out + ".kernelinput", 'w') as kernelfile: 
            for i, (source, file) in enumerate(all_files): 
                with open(file) as fasta:
                    with open(file + ".blastPsiMat") as matfile: 
                        header = fasta.readline().strip().split(" ")
                        fid = header[0] + "_" + str(i)
                        head = " ".join(header[1:])
                        idfile.write(fid + "\n")
                        group = "positive" if file.find("possitiv") > 0 else "negative"
                        ids.append((source, fid, group))
                        kernelfile.write(fid + " " + head)
                        kernelfile.write(fasta.readline())
                        kernelfile.write("\n")
                        for l in matfile.readlines():
                            kernelfile.write(l)
                            
    sIds = ",".join((fid[1:] for _, fid, _ in ids))
    if files2 is None:
        arffiles = [open(out + ".arff", 'w')]
    else:
        arffiles = [open(out + ".train.arff", 'w'), open(out + ".test.arff", 'w')]
    for arffile in arffiles:
        arffile.write("@relation docs\n")
        arffile.write("@attribute identifier {" + sIds + "}\n")
        arffile.write("@attribute class {positive,negative}\n")
        arffile.write("@data\n")
    for source, fid, group in ids: 
        if source == 1:
            outfile = arffiles[0]
        else: outfile = arffiles[1]
        outfile.write(fid[1:] + "," + group + "\n")
    for arffile in arffiles:
        arffile.close()
    


In [23]:
for l in lens:
    train_path = "data/region{}".format(l)
    train_files = !find {train_path} -iname "*.fa"
    write_files(train_path + "/region{}".format(l), train_files)
    for test_method in methods:
        test_path = "data/testset{}.{}".format(l, test_method)
        test_files = !find {test_path} -iname "*.fa"
        random.seed(42)
        test_files = random.sample(test_files, 1000)
        all_files = train_files + test_files
        write_files(test_path + "/testset{}.{}".format(l, test_method), train_files, test_files)

    